# Algoritmos de optimización - Seminario<br>
**Nombre y Apellidos:** Alberto Rodriguez Arizaga  <br>
**Url:** https://github.com/brtIn05/03MIAR---Algoritmos-de-Optimizacion/tree/main/Seminario<br>
**Problema:**

> 2. Organizar los horarios de partidos de La Liga<br>


**Descripción del problema:**

Desde la La Liga de fútbol profesional se pretende organizar los horarios de los partidos de liga de cada jornada. Se conocen algunos datos que nos deben llevar a diseñar un algoritmo que realice la asignación de los partidos a los horarios de forma que maximice la audiencia.

Se dispone de 10 horarios diferentes y 20 equipos en 3 Categorias A,B y C (que tiene relación directa con la audiencia). Se proporciona la matriz de datos de audiencia en función de los enfrenmtamientos por categoria de equipos.

Si el horario no se realiza a las 20 horas se aplica una penalización. 

Por último, se aplica otra penalización de audiencia en base al número de partidos programados en la misma franja horaria.

In [173]:
teams = 20
slots = 10
Cat_A = 3
Cat_B = 11
Cat_C = 6

In [174]:
# Creamos matriz de audiencias
#             #A  B  C
audience = [ 
            [2, 1.3, 1],  
            [1.3, 0.9, 0.75], 
            [1, 0.75, 0.47], 
]

In [175]:
# Creamos lista de categorias de equipos en la cual cada equipo correspondera a un índice de esta lista

category = []

for i in range (teams):

    if i < Cat_A: # Categoría A
        category.append(0)
    elif i < Cat_B: # Categoría B
        category.append(1)
    else: # Categoría C
        category.append(2)

print(category)

[0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2]


In [176]:
category = [0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2]

In [177]:
penalty_slot = [0.4,0.55,0.7,0.8,1,0.45,0.75,0.85,1,0.4]
# penalty_slot = [1,1,1,1,1,1,1,1,1,1]

In [178]:
penalty_coincidence = [1, 1, 0.75,0.55,0.4,0.30,0.25,0.23,0.2,0.2]
# penalty_coincidence = {0:1, 1:1, 2:0.02,3:0.02,4:0.02,5:0.02,6:0.02,7:0.02,8:0.02,9:0.02}

In [179]:
def calculate_audience(slots_usage, penalty_slot, penalty_coincidence):
    """
    Función que calcula audiencia de una combinación dada de partidos y slots
    """
    audience = 0
    # Recorremos el diccionario de partidos asignados a horarios
    # Contamos su audiencia base, reducimos por tipo de horario y por coincidencia
    for slot, matches in slots_usage.items():
        for match in matches:
            audience += match[-1]*penalty_slot[slot]*penalty_coincidence[len(matches)]
    
    audience = round(audience,3)
    print("\n\n #### Audiencia obtenida: ", audience)
    
    return audience


In [180]:
# Definir las variables de decisión

def max_audienc_per_slot(teams, slots, category ,audience,penalty_slot,penalty_coincidence):
    """
    Función que asigna slots a cada partido utilizando un algoritmo voraz
    """
    # Inicializamos la lista de partidos usados
    teams_already_used = []

    # Inicializamos el diccionario donde colocaremos cada uno de los partidos calculados
    # Las llaves serán los horarios y los valores los equipos
    slots_usage = {slot:[] for slot in range(slots)}

    # Creamos la lista de posibles combinaciones de partidos
    # Utilizamos una tupla que contendrá el equipo 1 el equipo 2 y su audiencia en función de la categoria de ambos equipos
    # Por ejemplo los equipos 0, 1, 2 serán los 3 equipos de Categoria A (primeros tres elementos de la lista category
    # que se indexa luego para obtener la audiencia en la matriz de audiencias
    match_audience = [(team1, team2, audience[category[team1]][category[team2]])  for team1 in range(teams) for team2 in range(team1+1,teams) ]

    # Ordenamos los partidos por su audiencia
    match_audience_sorted = sorted(match_audience,key=lambda x: x[2], reverse=True)

    # Recorremos la lista de partidos ordenada por audiencia
    for index, match in enumerate(match_audience_sorted):

        # Seleccionamos la parte de la tupla que contiene el partido
        match_teams = list(match[:2])
        # Inicializamos las variables de horario y mejor horario y valor de mejor horario
        slot = 0
        best_slot_value = 0
        best_slot = 0

        # Verificamos si los equipos ya han sido utilizados para eviatar que un equipo juegue más de una vez por jornada
    
        if match_teams[0] not in teams_already_used and match_teams[1] not in teams_already_used:

            # Verficamos que no sean los últimos dos partidos, en cuyo caso 
            # asumimos que se programarán viernes y sabado

            if len(teams_already_used) < teams-4:

                # Los incluimos en la lista de partidos usados
                teams_already_used.extend(match_teams)

                # Recorremos todos los slots
                # Este bucle es la clave del algoritmo y analizará cual es el mejor slot (recorriendo todos ellos) en función
                # de la penalización del mismo y de la coincidencia de horarios que existe.
                # Almacenará dichos valores en las variables de mejor solución. Con ello nos aseguraremos
                # de que elegimos la mejor solución en cada etapa.
                while slot<slots-1:

                    slot_value = penalty_slot[slot]*penalty_coincidence[len(slots_usage[slot])+1]
                
                    if  slot_value > best_slot_value:

                        best_slot_value = slot_value
                        best_slot = slot
                
                    slot += 1

                # Añadimos el partido al mejor slot encontrado

                slots_usage[best_slot].append(match)

                # Printeamos la solución para ese partido

                print(match[:2],f" in slot {best_slot}")

            else:
                # En caso de que sean los dos últimos partidos se envían directamente
                # al horario del lunes o el viernes
                if len(slots_usage[0]) == 0:
                    teams_already_used.extend(match_teams)
                    slots_usage[0].append(match)
                    #audience_value += penalty_slot[0] * match[-1]
                    print(match[:2],f" in slot {0}")

                else:
                    teams_already_used.extend(match_teams)
                    slots_usage[9].append(match)
                    #audience_value += penalty_slot[9] * match[-1]
                    print(match[:2],f" in slot {9}")
                    break
                               
    # Creamos una variable informativa que cuente cuantos partidos hemos puesto en cada horario            
    slots_usage_qty = {slot:len(slots_usage[slot]) for slot in range  (slots)}

    # Devolvemos la solúción encontrada       
    print("\n #### Allocation result: ")
    print(slots_usage)
    print(slots_usage_qty)


    return slots_usage, slots_usage_qty



In [181]:
# Lanzamos el algoritmo con los datos del problema
slots_usage, slots_usage_qty = max_audienc_per_slot(teams, slots, category, audience, penalty_slot, penalty_coincidence)
audience_result = calculate_audience(slots_usage, penalty_slot, penalty_coincidence)


(0, 1)  in slot 4
(2, 3)  in slot 8
(4, 5)  in slot 7
(6, 7)  in slot 3
(8, 9)  in slot 4
(10, 11)  in slot 6
(12, 13)  in slot 8
(14, 15)  in slot 2
(16, 17)  in slot 0
(18, 19)  in slot 9

 #### Allocation result: 
{0: [(16, 17, 0.47)], 1: [], 2: [(14, 15, 0.47)], 3: [(6, 7, 0.9)], 4: [(0, 1, 2), (8, 9, 0.9)], 5: [], 6: [(10, 11, 0.9)], 7: [(4, 5, 0.9)], 8: [(2, 3, 1.3), (12, 13, 0.9)], 9: [(18, 19, 0.47)]}
{0: 1, 1: 0, 2: 1, 3: 1, 4: 2, 5: 0, 6: 1, 7: 1, 8: 2, 9: 1}


 #### Audiencia obtenida:  6.69
